### MountainCar-v0 Environment

In [1]:
import gymnasium as gym
env = gym.make('MountainCar-v0', render_mode='rgb_array')
obs, info = env.reset()
import mediapy

frames = []
c = 0
while True:
    c += 1
    # sample random action
    # action = env.action_space.sample()
    action = 1
    obs, rewards, terms, truncs, info = env.step(action)
    done = terms or truncs
    img = env.render()
    frames.append(img)
    if done:
        break
env.close()
mediapy.show_videos({'MountainCar-V0-RandomAgent':frames})

MountainCar-V0-RandomAgent This browser does not support the video tag.


In [9]:
env.action_space, action,type(action)

(Discrete(3), 1, numpy.int64)

### Minigrid with 5x5 map, no walls / maze at all

In [2]:
import minigrid
import gymnasium as gym
import mediapy

name = "MiniGrid-Empty-5x5-v0"
env = gym.make(name, render_mode="rgb_array")
observation, info = env.reset(seed=42)
frames = []
for c in range(1000):
   # action = policy(observation)  # User-defined policy function
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        observation, info = env.reset()
env.close()

mediapy.show_videos({name: frames})

MiniGrid-Empty-5x5-v0 This browser does not support the video tag.


In [2]:
import minigrid
import gymnasium as gym
import mediapy

name = "MiniGrid-FourRooms-v0"
env = gym.make(name, render_mode="rgb_array")
observation, info = env.reset(seed=42)
frames = []
for c in range(1000):
   # action = policy(observation)  # User-defined policy function
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        observation, info = env.reset()
env.close()

mediapy.show_videos({name: frames})

MiniGrid-FourRooms-v0 This browser does not support the video tag.


In [5]:
observation

{'image': array([[[8, 1, 0],
         [2, 5, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[1, 0, 0],
         [2, 5, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[1, 0, 0],
         [2, 5, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[1, 0, 0],
         [2, 5, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[1, 0, 0],
         [2, 5, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],
 
        [[2, 5, 0],
         [2, 5, 0],
         [2, 5, 0],
         [2, 5, 0],
         [2, 5, 0],
         [2, 5, 0],
         [2, 

In [4]:
env.observation_space['image']

Box(0, 255, (7, 7, 3), uint8)

#### Custom MiniGrid Environment: OneWayDoor env
1. Terminates a litte too soon
2. can be clarified a bit more?

In [1]:
import minigrid
import gymnasium as gym
import mediapy

name = "MiniGrid-OneWayDoor-v0"
env = gym.make(name, render_mode="rgb_array")
observation, info = env.reset(seed=42)
frames = []
for c in range(1000):
   # action = policy(observation)  # User-defined policy function
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        observation, info = env.reset()
env.close()

mediapy.show_videos({name: frames})

MiniGrid-OneWayDoor-v0 This browser does not support the video tag.


In [16]:
obs, info = env.reset()
obs['image'].shape

(7, 7, 3)

In [17]:
wenv = minigrid.wrappers.ImgObsWrapper(env)
obs, info = wenv.reset()
obs.shape

(7, 7, 3)

In [25]:
genv = GoalCondEnvWrapper(wenv, episode_length=20, is_image_based=True)


In [23]:
from typing import *
import numpy as np

class GoalCondEnvWrapper(gym.ObservationWrapper):
    r"""
    Convert the concatenated observation space in GCRL into a better format with
    dict observations.
    """

    episode_length: int
    is_image_based: bool
    create_kwargs: Mapping[str, Any]

    def __init__(self, env: gym.Env, episode_length: int, is_image_based: bool = True):
        super().__init__(gym.wrappers.TimeLimit(env.unwrapped, episode_length))
        if is_image_based:
            # Observation space assumption : shape is (7, 7, 3)
            single_ospace = gym.spaces.Box(
                low=np.full((7, 7, 3), 0),
                high=np.full((7, 7, 3), 255),
                dtype=np.uint8,
            )
        else:
            assert isinstance(env.observation_space, gym.spaces.Box)
            ospace: gym.spaces.Box = env.observation_space
            assert len(ospace.shape) == 1
            single_ospace = gym.spaces.Box(
                low=np.split(ospace.low, 2)[0],
                high=np.split(ospace.high, 2)[0],
                dtype=ospace.dtype,
            )
        self.observation_space = gym.spaces.Dict(dict(
            observation=single_ospace,
            achieved_goal=single_ospace,
            desired_goal=single_ospace,
        ))
        self.episode_length = episode_length
        self.is_image_based = is_image_based

    def observation(self, observation):
        o, g = np.split(observation, 2)
        if self.is_image_based:
            o = o.reshape(7, 7, 3)
            g = g.reshape(7, 7, 3) # Wait, g is the goal map? what does that mean?
        odict = dict(
            observation=o,
            achieved_goal=o,
            desired_goal=g,
        )
        return odict

In [6]:
env.unwrapped

In [3]:
# One-way door construction in minigrid will be interesting -- this will actually help evaluating
# -- the quqsi-metric function's power, mountain car is meaningful since gravity is in play, too (I guess)

#### Unfortunately, MiniWorld requires lesser than python 3.11 version (newest not compatible)

In [4]:


import gymnasium as gym
env = gym.make("MiniWorld-OneRoom-v0")
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     action = policy(observation)  # User-defined policy function
#     observation, reward, terminated, truncated, info = env.step(action)

#     if terminated or truncated:
#         observation, info = env.reset()
# env.close()

NameNotFound: Environment `MiniWorld-OneRoom` doesn't exist. Did you mean: `MiniGrid-LockedRoom`?

In [10]:
import gymnasium as gym
import mediapy as media
name = 'roundabout-v0'
env = gym.make(name, render_mode='rgb_array')
frames = []
obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    frames.append(env.render())
media.show_videos({name: frames}, fps=6)

roundabout-v0 This browser does not support the video tag.
